<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a227ee5d1b87a4d933eb47811d466fcdac9827edc78c538196c112cb8b1502e8
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-24 14:49:21
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -74.02 K (-0.52%)
Current PnL: -23.36 L (-15.31%)
CY Booked + Current PnL: -9.47 L (-6.21%)
-------------------
Total profit:  1.45 L
Total loss:  -24.82 L
-------------------
Total Booked + Current PnL: 17.51 L (13.94%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.75%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.77 L (62.33%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.15%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.03,-14.21,16.67,0.09,14413.0,-14317.0,86460.0,100.64,49.0,M-SC,4.25,253.0,-0.99,0.61,13.59,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.76,-9.68,10.74,0.02,16932.0,-16905.0,157654.0,140.51,69.0,M-SC,5.40,234.0,-1.00,1.11,37.13,OX40N,NTT,PAINTS
43,ITC,452.00,-0.78,-1.13,11.70,10.44,23151.0,-2264.0,197874.0,-40.74,44.0,X-LC,0.77,5.0,-0.10,1.39,4.37,X40,NTT,FMCG
53,NESTLEIND,1377.00,-0.45,12.40,8.01,21.40,23896.0,32901.0,298327.0,4.29,58.0,X-LC,7.92,12.0,1.38,2.09,19.86,XY25,NTT,FMCG
35,ICICIGI,2252.93,-0.85,6.01,13.38,20.19,24725.0,10471.0,184791.0,-16.96,48.0,X-MC,7.60,68.0,0.42,1.30,22.03,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,CAMPUS,393.00,2.57,-9.25,46.86,33.28,71351.0,-15511.0,152264.0,-26.89,44.0,M-SC,2.95,222.0,-0.22,1.07,18.02,XY24,NTT,FOOTWEAR
57,RAJESHEXPO,518.00,2.57,-63.12,171.32,0.07,86678.0,-86583.0,50594.0,1754.80,60.0,L-SC,1.78,268.0,-1.00,0.36,25.83,OX40N,NTT,JEWELLERY
34,HONAUT,58357.33,2.31,-14.23,67.09,43.32,93725.0,-23172.0,139700.0,-26.34,39.0,X-SC,1.95,90.0,-0.25,0.98,7.26,X40N,ATH,ELECTRICAL
56,QUESS,424.00,2.01,-27.88,98.08,42.84,45979.0,-18127.0,46879.0,-52.62,40.0,X-SC,37.22,83.0,-0.39,0.33,2.01,XY24,NTT,MISC
29,HAPPSTMNDS,1480.71,1.67,-36.93,193.09,84.85,156079.0,-47333.0,80832.0,-25.05,48.0,H-SC,14.25,147.0,-0.30,0.57,3.74,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.00,-4.02,-46.42,225.06,74.15,197270.0,-75953.0,87652.0,-257.83,25.0,H-SC,14.61,149.0,-0.39,0.62,12.36,XR,NTT,CHEMICALS
72,TANLA,1943.92,-3.55,-37.04,248.94,119.69,424375.0,-100288.0,170473.0,-36.34,26.0,H-SC,12.00,156.0,-0.24,1.20,34.67,AR,ATH,IT
58,RAJOOENG,143.10,-3.46,-30.07,99.64,39.61,71422.0,-30820.0,71680.0,-52.76,20.0,H-SC,19.29,135.0,-0.43,0.50,0.00,AR,ATH,MISC
48,KPIGREEN,730.66,-3.20,-12.51,67.97,46.95,74509.0,-15677.0,109620.0,-19.22,26.0,H-SC,6.21,154.0,-0.21,0.77,37.33,MH,ATH,POWER
65,SHALBY,327.00,-3.19,-19.55,55.51,25.10,80891.0,-35419.0,145724.0,971.77,27.0,M-SC,16.44,250.0,-0.44,1.02,16.78,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.44,-0.82,108.43,106.72,132636.0,-1012.0,122324.0,-50.82,32.0,H-SC,4.06,139.0,-0.01,0.86,22.37,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.48,-5.41,119.80,107.91,167314.0,-7983.0,139661.0,-22.98,32.0,M-SC,10.37,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.76,-9.68,10.74,0.02,16932.0,-16905.0,157654.0,140.51,69.0,M-SC,5.40,234.0,-1.00,1.11,37.13,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.03,-14.21,16.67,0.09,14413.0,-14317.0,86460.0,100.64,49.0,M-SC,4.25,253.0,-0.99,0.61,13.59,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.34,-22.57,46.55,13.47,97196.0,-60867.0,208800.0,-68.12,27.0,H-SC,1.52,158.0,-0.63,1.47,6.17,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.15,-23.21,71.92,32.03,72813.0,-30593.0,101242.0,-16.88,47.0,H-SC,6.65,148.0,-0.42,0.71,13.40,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.58,1.87,69.11,72.27,119291.0,3162.0,172610.0,-12.43,58.0,M-LC,4.59,99.0,0.03,1.21,8.16,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.58,1.87,69.11,72.27,119291.0,3162.0,172610.0,-12.43,58.0,M-LC,4.59,99.0,0.03,1.21,8.16,XR,NTT,IT
38,INDIAMART,4810.62,-1.44,-0.82,108.43,106.72,132636.0,-1012.0,122324.0,-50.82,32.0,H-SC,4.06,139.0,-0.01,0.86,22.37,AR,ATH,MISC
25,FINCABLES,1641.55,-1.48,-5.41,119.80,107.91,167314.0,-7983.0,139661.0,-22.98,32.0,M-SC,10.37,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,0.09,-5.32,92.14,81.92,100146.0,-6109.0,108689.0,-44.55,32.0,M-SC,5.32,236.0,-0.06,0.76,28.67,XR,NTT,DURABLES
86,ZYDUSLIFE,1286.17,0.02,-2.51,39.12,35.63,79928.0,-5260.0,204314.0,-16.30,32.0,H-MC,4.51,119.0,-0.07,1.43,13.74,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.70,-14.76,66.81,42.18,157407.0,-40806.0,235604.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
46,JSWINFRA,342.0,-0.61,-9.62,27.49,15.23,49776.0,-19278.0,181069.0,-22.76,24.0,X-MC,6.50,66.0,-0.39,1.27,19.53,X40N,NTT,REALTY
11,BATAINDIA,2096.0,0.75,-34.98,107.96,35.20,90315.0,-45014.0,83656.0,9.75,26.0,X-SC,12.61,92.0,-0.50,0.59,0.75,X40,NTT,FOOTWEAR
60,RELAXO,1176.0,-0.94,-47.60,190.66,52.31,145026.0,-69095.0,76065.0,-44.25,28.0,X-SC,3.79,91.0,-0.48,0.53,1.82,X40N,NTT,FOOTWEAR
27,GLAXO,3466.2,-2.23,-1.96,41.93,39.15,81921.0,-3908.0,195376.0,-21.29,28.0,X-MC,9.06,60.0,-0.05,1.37,26.47,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.70,-14.76,66.81,42.18,157407.0,-40806.0,235604.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.78,-1.13,11.70,10.44,23151.0,-2264.0,197874.0,-40.74,44.0,X-LC,0.77,5.0,-0.10,1.39,4.37,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.00,1.28,29.32,30.97,52481.0,2256.0,178994.0,-22.52,49.0,X-MC,1.54,75.0,0.04,1.26,14.00,X40,ATH,INSURANCE
3,ACC,3906.0,-0.98,-23.84,115.54,64.15,209382.0,-56731.0,181220.0,-55.44,38.0,X-SC,1.93,82.0,-0.27,1.27,1.65,XY24,BTT,CEMENT
66,SIEMENS,4671.5,0.05,-14.69,47.13,25.51,74819.0,-27345.0,158750.0,-2.66,54.0,H-LC,1.93,49.0,-0.37,1.11,16.99,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-0.70,-14.76,66.81,42.18,157407.0,-40806.0,235604.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.26,-13.46,71.55,48.45,124148.0,-26994.0,173512.0,-19.66,32.0,X-MC,7.46,64.0,-0.22,1.22,0.26,X40N,ATH,FINANCE
55,PGHH,17907.65,-0.32,-4.32,39.81,33.76,76489.0,-8685.0,192135.0,-32.51,34.0,X-MC,3.20,57.0,-0.11,1.35,0.65,X40,ATH,FMCG
54,PAGEIND,51605.07,-0.12,-5.12,32.87,26.07,51067.0,-8380.0,155360.0,-30.18,35.0,X-MC,8.87,55.0,-0.16,1.09,0.71,X40,ATH,APPARELS
11,BATAINDIA,2096.00,0.75,-34.98,107.96,35.20,90315.0,-45014.0,83656.0,9.75,26.0,X-SC,12.61,92.0,-0.50,0.59,0.75,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,2.01,-27.88,98.08,42.84,45979.0,-18127.0,46879.0,-52.62,40.0,X-SC,37.22,83.0,-0.39,0.33,2.01,XY24,NTT,MISC
60,RELAXO,1176.00,-0.94,-47.60,190.66,52.31,145026.0,-69095.0,76065.0,-44.25,28.0,X-SC,3.79,91.0,-0.48,0.53,1.82,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.75,-34.98,107.96,35.20,90315.0,-45014.0,83656.0,9.75,26.0,X-SC,12.61,92.0,-0.50,0.59,0.75,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.59,3.05,20.40,24.08,26688.0,3875.0,130825.0,-5.33,45.0,X-SC,4.74,89.0,0.15,0.92,23.27,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-1.31,-5.15,28.19,21.58,37470.0,-7218.0,132921.0,-52.17,32.0,X-MC,7.03,56.0,-0.19,0.93,17.68,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.45,-14.01,38.71,19.28,115159.0,-48465.0,297491.0,-24.92,65.0,X-LC,7.63,1.0,-0.42,2.09,9.98,X40,ATH,IT
41,INFY,2275.00,0.61,5.82,46.36,54.87,153492.0,18199.0,331087.0,-17.44,62.0,X-LC,3.16,2.0,0.12,2.32,14.58,X40,BTT,IT
76,TMPV,600.00,-0.70,-14.76,66.81,42.18,157407.0,-40806.0,235604.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
82,VBL,671.64,-0.13,-9.70,50.24,35.67,143294.0,-30624.0,285218.0,-20.46,41.0,X-LC,6.05,4.0,-0.21,2.00,2.82,X40N,ATH,FMCG
43,ITC,452.00,-0.78,-1.13,11.70,10.44,23151.0,-2264.0,197874.0,-40.74,44.0,X-LC,0.77,5.0,-0.10,1.39,4.37,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.03,-13.66,110.41,81.67,89429.0,-12813.0,80997.0,7283.33,49.0,L-SC,18.56,271.0,-0.14,0.57,56.85,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.03,-12.98,60.36,39.54,53132.0,-13134.0,88025.0,-31.96,66.0,M-SC,6.81,220.0,-0.25,0.62,24.12,AR,ATH,AUTO
51,MASFIN,398.61,-2.13,-4.99,28.46,22.05,26493.0,-4890.0,93090.0,-18.25,50.0,H-SC,7.21,164.0,-0.18,0.65,35.31,XR,ATH,FINANCE
70,SURYODAY,216.00,0.35,-19.37,52.60,23.03,75944.0,-34690.0,144381.0,57.89,49.0,H-SC,11.13,167.0,-0.46,1.01,43.25,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.76,-9.68,10.74,0.02,16932.0,-16905.0,157654.0,140.51,69.0,M-SC,5.40,234.0,-1.00,1.11,37.13,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.76,-9.68,10.74,0.02,16932.0,-16905.0,157654.0,140.51,69.0,M-SC,5.40,234.0,-1.00,1.11,37.13,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.24,0.78,20.01,20.95,50773.0,1971.0,253739.0,-0.21,77.0,X-LC,12.75,31.0,0.04,1.78,36.70,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-0.64,-37.90,114.01,32.89,54660.0,-29264.0,47943.0,-692.31,61.0,L-MC,5.59,259.0,-0.54,0.34,32.03,XR,NTT,BANKS
0,5PAISA,593.00,-2.29,-36.22,77.01,12.90,107321.0,-79136.0,139360.0,105.53,56.0,H-SC,14.57,161.0,-0.74,0.98,15.88,OX40N,NTT,FINANCE
50,LTIM,7201.88,0.32,8.68,21.14,31.66,56555.0,21374.0,267525.0,4.56,63.0,H-LC,12.50,43.0,0.38,1.88,48.44,X200,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.83
1,25,44.70
2,50,75.45


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.53
MC    29.26
LC    25.19
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.87
X40      21.24
X40N     11.88
XR        9.63
XY25      9.08
AR        8.94
OX40N     7.70
X200      1.88
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.90
X-MC    22.85
X-LC    20.02
M-SC    12.01
X-SC     8.21
H-MC     4.70
H-LC     2.99
L-SC     1.41
M-MC     1.37
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.82,-5.04,39.01
IT,13.85,-14.61,73.78
FINANCE,9.42,-15.52,64.33
MISC,7.39,-23.34,73.87
PAINTS,5.99,-8.26,24.34
ELECTRICAL,5.91,-11.37,52.16
INSURANCE,4.42,-0.56,35.99
PHARMA,3.84,-2.09,34.76
AUTO,3.35,-20.49,72.16


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3066983.0,21
AR,1297970.0,10
XR,1290327.0,13
X40,1013411.0,14
X40N,755335.0,9
OX40N,696283.0,10
XY25,346134.0,6
SR,279829.0,2
MH,74509.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3575922.0,25
M-SC,1347776.0,15
X-MC,1229234.0,16
X-LC,908071.0,11
X-SC,750576.0,8
H-MC,408603.0,3
L-SC,260778.0,3
H-LC,131374.0,2
M-LC,119291.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1223942.0      6
           AR         881542.0      5
           XR         792855.0      7
M-SC       XY24       787488.0      6
X-MC       X40        448904.0      7
X-LC       X40        390242.0      5
X-MC       XY24       350719.0      3
H-SC       OX40N      323245.0      4
X-SC       X40N       294262.0      3
M-SC       OX40N      286360.0      5
X-SC       XY24       282049.0      3
H-SC       SR         279829.0      2
X-MC       X40N       247935.0      4
X-LC       XY24       231284.0      2
H-MC       AR         217102.0      2
X-LC       X40N       213138.0      2
H-MC       XY24       191501.0      1
X-MC       XY25       181676.0      2
X-SC       X40        174265.0      2
L-SC       XR         174100.0      2
M-SC       XR         149421.0      2
           AR         124507.0      2
M-LC       XR         119291.0      1
L-SC       OX40N       86678.0      1
H-LC       AR          74819.0      1
H-SC       MH          74509.0      1
X-LC       XY25        73407.0      2
H-LC       X200        56555.0      1
L-MC       XR          54660.0      1
M-MC       XY25        49825.0      1
L-LC       XY25        41226.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
